In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')
    
# csv_to_parquet('./train.csv', 'train')
# csv_to_parquet('./test.csv', 'test')

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

train Done.


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

test Done.


In [37]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [38]:
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

Done.
Done.


In [39]:
train = train.dropna()
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

Done.


In [46]:
train['Delay'] = le.fit_transform(train['Delay'])

In [47]:
train_x = train.drop(columns=['ID', 'Delay_num'])
train_x = train_x.reset_index(drop=True)

# train_y = train['Delay_num']
# train_y = train_y.reset_index(drop=True)

test_x = test.drop(columns=['ID'])

In [48]:
target = 'Delay'
features = [c for c in train_x.columns if c not in [target]]

In [ ]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

kf = KFold(n_splits = 10, random_state = 41, shuffle = True)
models = []
val_scores = []
preds = []

params = {
'n_estimators':5000,
'max_depth':6,
'early_stopping_rounds':200,
'learning_rate':0.01,
'one_hot_max_size':3,
'bootstrap_type':"MVS",
'l2_leaf_reg':2,
'random_state':41
         }

# bundle = [CatBoostClassifier, LGBMClassifier, XGBClassifier]
# for model in bundle:
# model = CatBoostClassifier(**params)
for i, (train_index, val_index) in tqdm(enumerate(kf.split(train, train[target]))):

        # clf.fit(train_x, train_y)

        X_train, X_val = train_x[features].loc[train_index], train_x[features].loc[val_index]
        y_train, y_val = train_x[target][train_index], train_x[target][val_index]

        model = CatBoostClassifier(**params)

        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 2000)

        models.append(model)
        score = log_loss(y_val, model.predict_proba(X_val))

        val_scores.append(score)

0it [00:00, ?it/s]

0:	learn: 0.6792473	test: 0.6791985	best: 0.6791985 (0)	total: 55.3ms	remaining: 13m 50s
2000:	learn: 0.1788661	test: 0.1780019	best: 0.1780019 (2000)	total: 1m 24s	remaining: 9m 6s
4000:	learn: 0.1772315	test: 0.1778021	best: 0.1777994 (3961)	total: 2m 48s	remaining: 7m 42s


1it [03:01, 181.61s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.1777960993
bestIteration = 4103

Shrink model to first 4104 iterations.
0:	learn: 0.6792394	test: 0.6792143	best: 0.6792143 (0)	total: 48.3ms	remaining: 12m 4s
2000:	learn: 0.1786956	test: 0.1793132	best: 0.1793131 (1997)	total: 1m 23s	remaining: 9m
4000:	learn: 0.1770489	test: 0.1790760	best: 0.1790751 (3932)	total: 2m 48s	remaining: 7m 43s


2it [06:55, 212.24s/it]

...

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.1824548391
bestIteration = 6055

Shrink model to first 6056 iterations.
0:	learn: 0.6792410	test: 0.6792202	best: 0.6792202 (0)	total: 51ms	remaining: 12m 44s
2000:	learn: 0.1786836	test: 0.1794035	best: 0.1794035 (2000)	total: 1m 23s	remaining: 9m 2s
4000:	learn: 0.1770404	test: 0.1791585	best: 0.1791581 (3999)	total: 2m 47s	remaining: 7m 41s


8it [28:16, 224.60s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.1791189409
bestIteration = 5197

Shrink model to first 5198 iterations.
0:	learn: 0.6792407	test: 0.6792141	best: 0.6792141 (0)	total: 45.6ms	remaining: 11m 24s
2000:	learn: 0.1786816	test: 0.1796655	best: 0.1796655 (2000)	total: 1m 23s	remaining: 9m 3s
4000:	learn: 0.1769892	test: 0.1795469	best: 0.1795460 (3926)	total: 2m 47s	remaining: 7m 41s


9it [31:26, 213.57s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.1795408861
bestIteration = 4287

Shrink model to first 4288 iterations.
0:	learn: 0.6792524	test: 0.6792036	best: 0.6792036 (0)	total: 49.3ms	remaining: 12m 19s
2000:	learn: 0.1789335	test: 0.1773096	best: 0.1773096 (2000)	total: 1m 23s	remaining: 9m 2s
4000:	learn: 0.1773110	test: 0.1770530	best: 0.1770521 (3991)	total: 2m 47s	remaining: 7m 40s
6000:	learn: 0.1759303	test: 0.1769201	best: 0.1769201 (6000)	total: 4m 11s	remaining: 6m 17s


In [ ]:
val_scores

In [52]:
cat_preds = []
for model in models[:10]:
    preds = model.predict_proba(test[features])
    cat_preds.append(preds)

In [62]:
pred = np.array(cat_preds).mean(axis=0)

submission = pd.DataFrame(data=pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('baseline_submission.csv', index=True)